# Build and train a simple model

This is a notebook that builds and trains a simple model as an example of how to use Sparana. It uses ReLu activations with a linear final layer, Xavier initialization and the Adam optimizer. The layers, initialization and optimizer are interchangable with some other types in the library.

In [1]:
# Numpy and cupy are imported, because I use them in all of my experiments, you don't need them to run these cells
import numpy as np
import cupy as cp

# This is needed to load the MNIST files for training and testing
import pickle

# These are the Sparana objects needed to build a model
from sparana.model import model
from sparana.layers import full_relu_layer
from sparana.layers import full_linear_layer
# This is the optimizer used to train the model
from sparana.optimizer import adam_optimizer
# Load the data into this object, which will return minibatches, and track how many minibatches/epochs 
# have been loaded
from sparana.data_loader import loader

# This is not needed to train and test a simple model, but I am going to demonstrate it here too
from sparana.saver import model_saver
# Put your own path in here
path = 'path'

# This initializes the model object, the 2 things that are required are the input size, and a list of layer objects.
mymodel = model(input_size = 784, 
                # These are the layers input as a list, the final layer size is the number of classes the model will have.
                layers = [full_relu_layer(size = 1000), 
                          full_relu_layer(size = 800),
                          full_relu_layer(size = 400),
                          full_linear_layer(size = 10)],
                # This is set automatically, but I keep it in here as a demonstration
                comp_type = 'GPU',
               #dropout = 0.5
               )

# Initialize the weights here, after this randomly generated matrices are now in the GPU memory
mymodel.initialize_weights('Xavier', bias_constant = 0.1)

# Initialize the Adam optimizer, the associated matrices are now in GPU memory
opt = adam_optimizer(mymodel, 0.0001, epsilon = 0.001)

#Initialize the saver
mysaver = model_saver(mymodel)

# Initialize the loader object and load the MNIST dataset from pickle files using pickle
myloader = loader(pickle.load(open('MNIST_train_images.p', 'rb')),
                 pickle.load(open('MNIST_train_labels.p', 'rb')),
                 pickle.load(open('MNIST_test_images.p', 'rb')),
                 pickle.load(open('MNIST_test_labels.p', 'rb')))


Initalizing GPU weights


# Training

This cell just trains, then stores the model on RAM. The saver object can save the model to the hard disk with the line:
```python
mysaver.pickle_model('filename.p')
```

Then instead of running the loop cell I run one with the 
```python
mysaver.load_model('filename.p')
```

There are other notebooks where I do that.

In [2]:
# This is 20,000 training steps 
for i in range(20000):
    images, labels = myloader.minibatch(250)
    opt.train_step(images, labels)    

print(mymodel.get_accuracy(myloader.test_data(), myloader.test_labels()))

mysaver.store_model()

myloader.print_stats()

0.9839
Epochs:  90
Minibatches:  20000
